# RDD

an RDD is the fundamental data structure of Apache Spark. It provised low level data abstraction. It's a fault-tolerant, distributed collection of elements that can be operated on in parallel.

**Key Characteristics:**

- Immutable
- Lazy evaluation
- Fault tolerant (via lineage info)
- Partitioned across cluster nodes
- Can be cached in memory

### SparkContext and SparkConf


SparkContext is the foundational entry point for any Apache Spark application. It serves as the primary connection to a Spark cluster and is responsible for managing resources and orchestrating distributed computations.

**Key Aspects:**

- Cluster Connection: It establishes and maintains the connection between your Spark application (the driver program) and the Spark cluster
- Resource Management: SparkContext interacts with the cluster manager to acquire and manage the computational resources (executors, memory, CPU cores) needed for your application's tasks.
- RDD Creation: SparkContext was the primary way to create Resilient Distributed Datasets (RDDs), which are the fundamental low-level data abstraction in Spark.

#### `SparkConf`

SparkConf is a configuration class used to set various Spark properties and parameters for a Spark application. It allows you to define settings such as the application name, the master URL (where the Spark cluster is located), memory allocations for executors, number of CPU cores, and other specific configurations that control Spark's behavior. A SparkConf object is typically used to initialize a SparkContext or SparkSession, providing the necessary environment details for the application to run on the cluster.

**Common settings:**

- setMaster("local[*]") – Use local mode with all cores
- setAppName("RDDExample") – Application name

### transformations

Transformations create a new RDD from an existing one. They are lazy – not executed until an action is triggered. They are characterized by their lazy evaluation, meaning they do not immediately execute when called. Instead, Spark builds a Directed Acyclic Graph (DAG) of the computations, which represents the lineage of transformations. 

| Transformation  | Description                                          |
| --------------- | ---------------------------------------------------- |
| `map(func)`     | Returns a new RDD by applying `func` to each element |
| `filter(func)`  | Filters elements for which `func` returns true       |
| `flatMap(func)` | Like map but flattens the result                     |
| `distinct()`    | Removes duplicates                                   |
| `union(rdd)`    | Combines two RDDs                                    |
| `groupByKey()`  | Groups values with same key                          |
| `reduceByKey()` | Aggregates values with same key using a function     |
| `sortBy(func)`  | Sorts RDD by computed key                            |


### actions

Actions trigger the execution of the entire DAG of transformations and return a result to the driver program or write data to an external storage system. They are eagerly evaluated, meaning they execute immediately when called. 

| Action             | Description                            |
| ------------------ | -------------------------------------- |
| `collect()`        | Returns all elements to driver         |
| `count()`          | Returns number of elements             |
| `first()`          | Returns first element                  |
| `take(n)`          | Returns first `n` elements             |
| `reduce(func)`     | Reduces elements using binary operator |
| `saveAsTextFile()` | Writes RDD to text files               |



reference - [spark rdd docs](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


In [2]:
! pip install pyspark


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
from pyspark import SparkConf, SparkContext

# standalone cluster mode
# conf = SparkConf().setAppName("demoApp").setMaster("spark://codespaces-c6070e:7077")

# local
conf = SparkConf().setAppName("demoApp").setMaster("local[*]")


sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/05 04:15:03 WARN Utils: Your hostname, codespaces-c6070e, resolves to a loopback address: 127.0.0.1; using 10.0.1.210 instead (on interface eth0)
25/08/05 04:15:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/05 04:15:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc.defaultParallelism

2

In [3]:
rdd = sc.parallelize([1,2,3,4])

rdd = rdd.map(lambda x: x*x)

In [4]:
rdd.collect()

[1, 4, 9, 16]

In [5]:
sc

<SparkContext master=local[*] appName=demoApp>

In [6]:
data = [
    "apple banana orange",
    "apple mango banana",
    "grapes pipneapple"
]

In [ ]:
lines = sc.parallelize(data)

lines.collect()

['apple banana orange', 'apple mango banana', 'grapes pipneapple']

In [8]:
# map

words = lines.map(lambda line: line.split())

words.collect()





[['apple', 'banana', 'orange'],
 ['apple', 'mango', 'banana'],
 ['grapes', 'pipneapple']]

In [9]:
# flatmap

words = lines.flatMap(lambda line: line.split())

words.collect()

['apple',
 'banana',
 'orange',
 'apple',
 'mango',
 'banana',
 'grapes',
 'pipneapple']

In [10]:
# key value pairs using map()

word_pairs = words.map(lambda word: (word, 1))

word_pairs.collect()

[('apple', 1),
 ('banana', 1),
 ('orange', 1),
 ('apple', 1),
 ('mango', 1),
 ('banana', 1),
 ('grapes', 1),
 ('pipneapple', 1)]

In [11]:
# count each word using  reduceByKey()

word_counts = word_pairs.reduceByKey(lambda a,b:a+b)

word_counts.collect()

[('apple', 2),
 ('banana', 2),
 ('pipneapple', 1),
 ('orange', 1),
 ('mango', 1),
 ('grapes', 1)]

In [ ]:
grouped = word_pairs.groupByKey()

# grouped.collect()
# groupBy is more expensive than reduceByKey()

print([(k, list(v)) for k,v in grouped.collect()] )




[('apple', [1, 1]), ('banana', [1, 1]), ('pipneapple', [1]), ('orange', [1]), ('mango', [1]), ('grapes', [1])]


In [ ]:
# mapValues() -> used to only transform values

squared_counts = word_counts.mapValues(lambda x: x*x)

squared_counts.collect()

[('apple', 4),
 ('banana', 4),
 ('pipneapple', 1),
 ('orange', 1),
 ('mango', 1),
 ('grapes', 1)]

In [ ]:
# sortByKey - sort words alphabetically

sorted_kv = word_counts.sortByKey()

sorted_kv.collect()

[('apple', 2),
 ('banana', 2),
 ('grapes', 1),
 ('mango', 1),
 ('orange', 1),
 ('pipneapple', 1)]

---

#### working with sales data

relative path: `datasets/region_sales.txt`

In [32]:
# lodaing the data from a file

sales_raw = sc.textFile("file:///workspaces/trng-2286/datasets/region_sales.txt")

In [33]:
sales_raw.take(9)

['2001,South,Electronics,838',
 '2002,West,Furniture,471',
 '2003,North,Electronics,803',
 '2004,West,Furniture,174',
 '2005,South,Clothing,590',
 '2006,North,Furniture,937',
 '2007,North,Electronics,391',
 '2008,West,Electronics,961',
 '2009,North,Electronics,305']

In [34]:
records = sales_raw.map(lambda x:x.split(","))

records.take(4)

[['2001', 'South', 'Electronics', '838'],
 ['2002', 'West', 'Furniture', '471'],
 ['2003', 'North', 'Electronics', '803'],
 ['2004', 'West', 'Furniture', '174']]

In [35]:
records = records.map(lambda x: (int(x[0]), x[1], x[2], int(x[3])))

records.take(9)

[(2001, 'South', 'Electronics', 838),
 (2002, 'West', 'Furniture', 471),
 (2003, 'North', 'Electronics', 803),
 (2004, 'West', 'Furniture', 174),
 (2005, 'South', 'Clothing', 590),
 (2006, 'North', 'Furniture', 937),
 (2007, 'North', 'Electronics', 391),
 (2008, 'West', 'Electronics', 961),
 (2009, 'North', 'Electronics', 305)]

In [36]:
# total sales by categroy

category_sales = records.map(lambda x: (x[2], x[3]))

category_sales.take(4)

[('Electronics', 838),
 ('Furniture', 471),
 ('Electronics', 803),
 ('Furniture', 174)]

In [37]:
total_sales_by_category = category_sales.reduceByKey(lambda x,y: x+y)

total_sales_by_category.collect()

[('Furniture', 12832),
 ('Clothing', 14410),
 ('Electronics', 15593),
 ('Books', 10739)]

In [ ]:
# average sales per category

average_sales_per_category = category_sales.mapValues(lambda x:(x,1)) \
                        .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1] )) \
                        .mapValues(lambda x: x[0]/x[1])


average_sales_per_category.collect()

[('Furniture', 583.2727272727273),
 ('Clothing', 514.6428571428571),
 ('Electronics', 556.8928571428571),
 ('Books', 488.1363636363636)]

In [43]:
# highest selling  category

highest_selling_category = total_sales_by_category.max(key=lambda x: [x[1]])

highest_selling_category

('Electronics', 15593)

In [44]:
# categories with sales above 13000

highest_selling_categories_above_13k = total_sales_by_category.filter(lambda x: x[1]>13000)

highest_selling_categories_above_13k.collect()



[('Clothing', 14410), ('Electronics', 15593)]

### Shared Variables

When you pass a function (like in map or reduce) to Spark, that function runs on different machines in the cluster — not on your local driver.

**By default:**

- Spark makes separate copies of any variable you use inside those functions.
- So, if a task changes a variable on the executor, that change will not reflect back in your driver program.

This is done to keep things fast and distributed — but it also means you can not just update normal variables across tasks.

**challenges:**

You want to count how many rows have Amount > 500 using this code:

```py
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[2]) > 500).foreach(increment_count)
print(count)

```

This will not work because each machine updates its own copy of count not the original one in the driver.

To solve such issues, spark provides two types of shared variables:

1. **Broadcast Variables:** a read-only variable that can be cached on each machine (executor). Used to efficiently share large data (like lookup tables) with all tasks without copying it multiple times.
2. **Accumulators:** variables used to safely implement counters or sums across mulitple worker nodes. You can only add to them (not read or subtract inside tasks). The final value is only accessible on the driver after an action is executed.

Spark shared variables are a mechanism for efficiently distributing read-only data or aggregating results across executors in a Spark application.

In [ ]:
# this fails

count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[3]) > 500).foreach(increment_count)
print(count)

In [47]:
# broadcast variable

regions = {"North": "N", "South": "S", "East": "E", "West": "W"}
region_broadcast = sc.broadcast(regions)


In [48]:
# accumulator 

high_value_count = sc.accumulator(0)

In [49]:
def enrich_region_and_count(record):
    pid, region, category, ammount = record
    if ammount > 500:
        high_value_count.add(1)
    return (pid, region_broadcast.value[region], category, ammount)

enriched_records = records.map(enrich_region_and_count)

enriched_records.collect()

[(2001, 'S', 'Electronics', 838),
 (2002, 'W', 'Furniture', 471),
 (2003, 'N', 'Electronics', 803),
 (2004, 'W', 'Furniture', 174),
 (2005, 'S', 'Clothing', 590),
 (2006, 'N', 'Furniture', 937),
 (2007, 'N', 'Electronics', 391),
 (2008, 'W', 'Electronics', 961),
 (2009, 'N', 'Electronics', 305),
 (2010, 'E', 'Clothing', 213),
 (2011, 'E', 'Electronics', 615),
 (2012, 'S', 'Clothing', 573),
 (2013, 'E', 'Clothing', 352),
 (2014, 'W', 'Clothing', 768),
 (2015, 'N', 'Electronics', 231),
 (2016, 'E', 'Furniture', 217),
 (2017, 'W', 'Clothing', 346),
 (2018, 'N', 'Books', 375),
 (2019, 'S', 'Electronics', 313),
 (2020, 'W', 'Furniture', 903),
 (2021, 'W', 'Clothing', 904),
 (2022, 'N', 'Furniture', 812),
 (2023, 'W', 'Furniture', 590),
 (2024, 'N', 'Books', 452),
 (2025, 'N', 'Books', 697),
 (2026, 'E', 'Clothing', 959),
 (2027, 'N', 'Books', 661),
 (2028, 'S', 'Books', 700),
 (2029, 'E', 'Clothing', 832),
 (2030, 'N', 'Furniture', 163),
 (2031, 'S', 'Books', 413),
 (2032, 'N', 'Furniture',

In [50]:
high_value_count.value

51

In [51]:
# rdd to a new file

filtered = enriched_records.filter(lambda x:x[3]> 500)

filtered.map(lambda x: ",".join(map(str,x))).coalesce(1).saveAsTextFile("filtered_high_value_sales")

In [52]:
filtered = enriched_records.filter(lambda x:x[3]> 500)

filtered.map(lambda x: ",".join(map(str,x))).repartition(2).saveAsTextFile("filtered_high_value_sales_repartition")

# Activity


1. Find all product IDs where the amount is greater than 900.
2. Find all transactions that belong to the “Furniture” category.
3. Count how many transactions belong to the “Electronics” category.
4. Find average amount for each category.
5. Find the highest amount and the corresponding product ID.
6. Find the total number of unique categories.
7. For each category, find the product ID with the highest sale.
8. Count how many products were sold for less than 300.
9. Sort the transactions by amount in descending order.
10. Broadcast Variable: Category Discounts
```{"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}```
11. Accumulator: Count Transactions Below 300
12. Filter and Save Results